In [50]:
!head data/smsspamcollection.csv


ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam	H

In [51]:
import pandas as pd
import numpy as np

pd.set_option('max_colwidth', 600)

SPAM = 1
HAM = 0
ABSTAIN = -1

df = pd.read_csv('data/smsspamcollection.csv', 
                 header=None,  
                 sep='\t', 
                 names = ['old_label', 'text'])
df.head()

,old_label,text
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


In [52]:
df['label'] = df['old_label'].apply(lambda x: SPAM if x == 'spam' else HAM )
df.head()

,old_label,text,label
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives around here though",0


In [53]:
df.loc[ df.sample(frac=0.8).index, 'label'] = ABSTAIN
df.drop(columns=['old_label'], inplace=True)
df.head()

,text,label
0,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",-1
1,Ok lar... Joking wif u oni...,-1
2,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,1
3,U dun say so early hor... U c already then say...,-1
4,"Nah I don't think he goes to usf, he lives around here though",-1


In [54]:
abstain_idx = df['label'] == ABSTAIN
abstain_idx

0        True
1        True
2       False
3        True
4        True
        ...  
5567     True
5568     True
5569     True
5570     True
5571     True
Name: label, Length: 5572, dtype: bool

In [55]:
df_train = df[abstain_idx]
df_test = df[~abstain_idx]

In [56]:
!pip install snorkel

# Simple keyword search


In [57]:
from snorkel.labeling import labeling_function

@labeling_function()
def check(sms):
    return SPAM if 'check' in sms.text.lower() else ABSTAIN

@labeling_function()
def free(sms):
    return SPAM if 'free' in sms.text.lower() else ABSTAIN



apply the labeling functions to the train set.

In [58]:
from snorkel.labeling import PandasLFApplier

lfs = [check, free]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)
L_train

  0%|          | 0/4458 [00:00<?, ?it/s]

100%|██████████| 4458/4458 [00:00<00:00, 22147.40it/s]


array([[-1, -1],
       [-1, -1],
       [-1, -1],
       ...,
       [-1, -1],
       [-1,  1],
       [-1, -1]])

The simplest way to analyze this is to determine the coverage of labeling functions (i.e., the percentage of cases for which the function returned a result other than `ABSTAIN'.



In [59]:
coverage_check, coverage_free = (L_train != ABSTAIN).mean(axis=0)

print(f"Coverage for check(): {coverage_check * 100:.1f}%")
print(f"Coverage for free(): {coverage_free * 100:.1f}%")


Coverage for check(): 1.0%
Coverage for free(): 4.6%


In [60]:
from snorkel.labeling import LFAnalysis

LFAnalysis(L=L_train, lfs=lfs).lf_summary()


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.010094,0.0,0.0
free,1,[1],0.046433,0.0,0.0


In [61]:
df_train.iloc[L_train[:,1] == SPAM].sample(frac=0.1)


,text,label
385,"Double mins and txts 4 6months FREE Bluetooth on Orange. Available on Sony, Nokia Motorola phones. Call MobileUpd8 on 08000839402 or call2optout/N9DX",-1
2514,U have won a nokia 6230 plus a free digital camera. This is what u get when u win our FREE auction. To take part send NOKIA to 83383 now. POBOX114/14TCR/W1 16,-1
4077,87077: Kick off a new season with 2wks FREE goals & news to ur mobile! Txt ur club name to 87077 eg VILLA to 87077,-1
1221,No. 1 Nokia Tone 4 ur mob every week! Just txt NOK to 87021. 1st Tone FREE ! so get txtin now and tell ur friends. 150p/tone. 16 reply HL 4info,-1
319,December only! Had your mobile 11mths+? You are entitled to update to the latest colour camera mobile for Free! Call The Mobile Update Co FREE on 08002986906,-1
3141,sexy sexy cum and text me im wet and warm and ready for some porn! u up for some fun? THIS MSG IS FREE RECD MSGS 150P INC VAT 2 CANCEL TEXT STOP,-1
744,Ok. There may be a free gym about.,-1
1518,Our brand new mobile music service is now live. The free music player will arrive shortly. Just install on your phone to browse content from the top artists.,-1
268,Ur ringtone service has changed! 25 Free credits! Go to club4mobiles.com to choose content now! Stop? txt CLUB STOP to 87070. 150p/wk Club4 PO Box1146 MK45 2WT,-1
4303,Aiyar hard 2 type. U later free then tell me then i call n scold n tell u.,-1


In [64]:
@labeling_function()
def call(sms):
    return SPAM if 'call' in sms.text.lower() else ABSTAIN

lfs = [check, free, call]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)
LFAnalysis(L=L_train, lfs=lfs).lf_summary()


100%|██████████| 4458/4458 [00:00<00:00, 21795.50it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.010094,0.000449,0.0
free,1,[1],0.046433,0.016599,0.0
call,2,[1],0.115747,0.017048,0.0


In [65]:
# examples were labeled as spam by the call_now() function but omitted by free().
from snorkel.analysis import get_label_buckets

buckets = get_label_buckets(L_train[:, 1], L_train[:, 2])
buckets


{(-1, -1): array([   0,    1,    2, ..., 4454, 4455, 4457]),
 (1,
  -1): array([   4,   11,   71,   75,  112,  137,  138,  201,  217,  219,  290,
         328,  342,  397,  425,  493,  511,  533,  545,  605,  648,  656,
         730,  774,  825,  886,  942,  991,  997,  999, 1037, 1097, 1106,
        1167, 1178, 1205, 1215, 1216, 1225, 1241, 1248, 1296, 1310, 1361,
        1371, 1423, 1428, 1439, 1529, 1537, 1547, 1558, 1586, 1605, 1613,
        1675, 1679, 1706, 1716, 1745, 1807, 1835, 1873, 1875, 1928, 1973,
        2008, 2037, 2046, 2072, 2088, 2093, 2129, 2158, 2267, 2308, 2317,
        2426, 2436, 2528, 2539, 2599, 2674, 2677, 2687, 2764, 2769, 2833,
        2937, 2951, 2981, 2996, 3036, 3068, 3100, 3156, 3172, 3268, 3282,
        3289, 3327, 3341, 3373, 3388, 3401, 3458, 3543, 3649, 3672, 3681,
        3785, 3789, 3822, 3893, 3925, 3928, 4003, 4047, 4080, 4110, 4118,
        4123, 4124, 4148, 4158, 4162, 4234, 4243, 4278, 4287, 4372, 4429,
        4456]),
 (-1,
  1): array([   6,

In [66]:
buckets.keys()

dict_keys([(-1, -1), (1, -1), (-1, 1), (1, 1)])

In [67]:
df_train.iloc[buckets[(SPAM, ABSTAIN)]]


,text,label
5,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv",-1
12,"URGENT! You have won a 1 week FREE membership in our £100,000 Prize Jackpot! Txt the word: CLAIM to No: 81010 T&C www.dbuk.net LCCLTD POBOX 4403LDNW1A7RW18",-1
89,"Ela kano.,il download, come wen ur free..",-1
95,"Your free ringtone is waiting to be collected. Simply text the password ""MIX"" to 85069 to verify. Get Usher and Britney. FML, PO Box 5249, MK17 92H. 450Ppw 16",-1
139,You'll not rcv any more msgs from the chat svc. For FREE Hardcore services text GO to: 69988 If u get nothing u must Age Verify with yr network & try again,-1
...,...,...
5342,"u r subscribed 2 TEXTCOMP 250 wkly comp. 1st wk?s free question follows, subsequent wks charged@150p/msg.2 unsubscribe txt STOP 2 84128,custcare 08712405020",-1
5353,Guai... Ü shd haf seen him when he's naughty... Ü so free today? Can go jogging...,-1
5467,Get your garden ready for summer with a FREE selection of summer bulbs and seeds worth £33:50 only with The Scotsman this Saturday. To stop go2 notxt.co.uk,-1
5540,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE MINS. INDIA CUST SERVs SED YES. L8ER GOT MEGA BILL. 3 DONT GIV A SHIT. BAILIFF DUE IN DAYS. I O £250 3 WANT £800,-1


In [68]:
@labeling_function()
def hot(sms):
    return SPAM if 'HOT' in sms.text else ABSTAIN


In [70]:
import re

@labeling_function()
def regex_I_am_free(sms):
    if re.search(r"\sI\s.*free", sms.text, flags=re.I):
        return HAM
    elif re.search(r"free", sms.text, flags=re.I):
        return SPAM
    else:
        return ABSTAIN

lfs = [check, free, call, regex_I_am_free]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 4458/4458 [00:00<00:00, 7424.91it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.010094,0.000449,0.000000
free,1,[1],0.046433,0.046433,0.002692
call,2,[1],0.115747,0.017048,0.000224
regex_I_am_free,3,"[0, 1]",0.046433,0.046433,0.002692


In [71]:
buckets = get_label_buckets(L_train[:, 1], L_train[:, 3])
df_train.iloc[buckets[(SPAM, HAM)]].sample(10, random_state=1)


,text,label
1784,No dear i do have free messages without any recharge. Hi hi hi,-1
2121,"Argh my 3g is spotty, anyway the only thing I remember from the research we did was that province and sterling were the only problem-free places we looked at",-1
2232,"K, wen ur free come to my home and also tel vikky i hav sent mail to him also.. Better come evening il be free today aftr 6pm..:-)",-1
4587,"Mila, age23, blonde, new in UK. I look sex with UK guys. if u like fun with me. Text MTALK to 69866.18 . 30pp/txt 1st 5free. £1.50 increments. Help08718728876",-1
1364,"Yetunde, i'm sorry but moji and i seem too busy to be able to go shopping. Can you just please find some other way to get what you wanted us to get. Please forgive me. You can reply free via yahoo messenger.",-1
3436,Hi darlin i cantdo anythingtomorrow as myparents aretaking me outfor a meal. when are u free? Katexxx,-1
948,Hey i booked the kb on sat already... what other lessons are we going for ah? Keep your sat night free we need to meet and confirm our lodging,-1
3692,"Sir, i am waiting for your call, once free please call me.",-1
5570,The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free,-1
4151,I only work from mon to thurs but Sat i cant leh... Booked liao... Which other day u free?,-1


In [72]:
@labeling_function()
def contains_money(sms):
    if re.search(r"\$\d+|£\d+", sms.text, flags=re.I):
        return SPAM
    else:
        return ABSTAIN


In [75]:
!python -m spacy download en_core_web_sm


/home/igor/.conda/envs/ml/lib/python3.11/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 8.3 MB/s eta 0:00:00m eta 0:00:010:0101
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [76]:
!python -m spacy validate


/home/igor/.conda/envs/ml/lib/python3.11/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.7.4) =================
ℹ spaCy installation:
/home/igor/.conda/envs/ml/lib/python3.11/site-packages/spacy

NAME             SPACY            VERSION                            
en_core_web_sm   >=3.7.2,<3.8.0   3.7.1   ✔



In [77]:
import spacy

nlp = spacy.load('en_core_web_sm')


/home/igor/.conda/envs/ml/lib/python3.11/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [78]:
_text = """I don't England is a country that is part of the United Kingdom. 
It shares land borders with Wales to its west and Scotland to its north. 
The Irish Sea lies northwest of England and the Celtic Sea to the southwest. 
England is separated from continental Europe by the North Sea to the east and the 
English Channel to the south. The country covers five-eighths of the island of 
Great Britain, which lies in the North Atlantic, and includes over 100 smaller islands, 
such as the Isles of Scilly and the Isle of Wight."""

doc = nlp(_text)

for e in doc.ents:
    print(f"{e.text:<25} start:{e.start} end:{e.end} label: {e.label_}")


England                   start:3 end:4 label: GPE
the United Kingdom        start:11 end:14 label: GPE
Wales                     start:21 end:22 label: ORG
Scotland                  start:26 end:27 label: GPE
The Irish Sea             start:32 end:35 label: LOC
England                   start:38 end:39 label: GPE
the Celtic Sea            start:40 end:43 label: LOC
England                   start:48 end:49 label: GPE
Europe                    start:53 end:54 label: LOC
the North Sea             start:55 end:58 label: LOC
English                   start:64 end:65 label: LANGUAGE
five-eighths              start:73 end:76 label: CARDINAL
Great Britain             start:81 end:83 label: GPE
the North Atlantic        start:87 end:90 label: LOC
over 100                  start:93 end:95 label: CARDINAL
the Isles of Scilly       start:101 end:105 label: GPE
the Isle of Wight         start:106 end:110 label: GPE


In [79]:
from snorkel.preprocess.nlp import SpacyPreprocessor

spacy = SpacyPreprocessor(text_field="text", doc_field="doc", memoize=True)


In [80]:
df_train.columns


Index(['text', 'label'], dtype='object')

In [81]:
@labeling_function(pre=[spacy])
def has_person(sms):
    if len(sms.doc) < 20 and any([ent.label_ == "PERSON" for ent in sms.doc.ents]):
        return HAM
    else:
        return ABSTAIN


In [82]:
lfs = [check, free, call, regex_I_am_free, has_person]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()


100%|██████████| 4458/4458 [00:48<00:00, 92.38it/s] 


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.010094,0.001570,0.001122
free,1,[1],0.046433,0.046433,0.003813
call,2,[1],0.115747,0.018170,0.001795
regex_I_am_free,3,"[0, 1]",0.046433,0.046433,0.003813
has_person,4,[0],0.051593,0.003365,0.003365


In [85]:
from wordfreq import zipf_frequency
from snorkel.preprocess import preprocessor

@preprocessor(memoize=True)
def avg_word_freq(sms):
    sms.avg_word_freq = sum([zipf_frequency(word, 'en') for word in sms.text.split()]) / len(sms.text.split())
    
    return sms

In [86]:
@labeling_function(pre=[avg_word_freq])
def many_rare_words(sms):
    return ABSTAIN if sms.avg_word_freq >= 4 else SPAM


In [88]:
lfs = [check, free, call, regex_I_am_free, has_person, many_rare_words]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()


100%|██████████| 4458/4458 [00:03<00:00, 1316.67it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.010094,0.001795,0.001122
free,1,[1],0.046433,0.046433,0.003813
call,2,[1],0.115747,0.027815,0.001795
regex_I_am_free,3,"[0, 1]",0.046433,0.046433,0.003813
has_person,4,[0],0.051593,0.011664,0.011664
many_rare_words,5,[1],0.064379,0.025123,0.009197


In [90]:
df_train.iloc[L_train[:,5] == SPAM].sample(frac=0.1)


,text,label
4821,Check Out Choose Your Babe Videos @ sms.shsex.netUN fgkslpoPW fgkslpo,-1
5413,Nite nite pocay wocay luv u more than n e thing 4eva I promise ring u 2morrowxxxx,-1
5319,Kothi print out marandratha.,-1
1466,"YOU 07801543489 are guaranteed the latests Nokia Phone, a 40GB iPod MP3 player or a £500 prize! Txt word:COLLECT to No:83355! TC-LLC NY-USA 150p/Mt msgrcvd18+",-1
2966,Not yet had..ya sapna aunty manege y'day hogidhe..chinnu full weak and swalpa black agidhane..,-1
5188,Okie,-1
2718,Okie.. Thanx..,-1
1007,Panasonic & BluetoothHdset FREE. Nokia FREE. Motorola FREE & DoubleMins & DoubleTxt on Orange contract. Call MobileUpd8 on 08000839402 or call 2optout,-1
4937,K..k.:)congratulation ..,-1
3358,"Brainless Baby Doll..:-D;-), vehicle sariyag drive madoke barolla..",-1


In [91]:
lfs = [check, free, call, regex_I_am_free, has_person, many_rare_words]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)
L_test = applier.apply(df=df_test)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()


100%|██████████| 1114/1114 [00:17<00:00, 65.31it/s] 


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.010094,0.001795,0.001122
free,1,[1],0.046433,0.046433,0.003813
call,2,[1],0.115747,0.027815,0.001795
regex_I_am_free,3,"[0, 1]",0.046433,0.046433,0.003813
has_person,4,[0],0.051593,0.011664,0.011664
many_rare_words,5,[1],0.064379,0.025123,0.009197


In [92]:
LFAnalysis(L=L_test, lfs=lfs).lf_summary()


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.013465,0.002693,0.000898
free,1,[1],0.052065,0.052065,0.004488
call,2,[1],0.108618,0.026032,0.000898
regex_I_am_free,3,"[0, 1]",0.052065,0.052065,0.004488
has_person,4,[0],0.041293,0.008079,0.008079
many_rare_words,5,[1],0.059246,0.020646,0.007181


In [93]:
from snorkel.labeling.model import MajorityLabelVoter

majority_model = MajorityLabelVoter()
preds_train = majority_model.predict(L=L_train)


In [94]:
preds_train


array([-1,  1, -1, ..., -1, -1, -1])

In [95]:
import numpy as np

labels, counts = np.unique(preds_train, return_counts=True)

for l, c in zip(labels, counts):
    print(f"LABEL: {l}, count: {c}")


LABEL: -1, count: 3430
LABEL: 0, count: 178
LABEL: 1, count: 850


In [96]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=42)


INFO:root:Computing O...
INFO:root:Estimating \mu...
100%|██████████| 500/500 [00:02<00:00, 249.48epoch/s]
INFO:root:Finished Training


In [97]:
majority_acc = majority_model.score(L=L_test, Y=df_test.label, tie_break_policy="random")["accuracy"]
print(f"{'Majority voting accuracy:':<25} {majority_acc * 100:.1f}%")

label_model_acc = label_model.score(L=L_test, Y=df_test.label, tie_break_policy="random")["accuracy"]
print(f"{'Probabilistic model accuracy:':<25} {label_model_acc * 100:.1f}%")


Majority voting accuracy: 48.3%
Probabilistic model accuracy: 48.9%


In [98]:
from snorkel.labeling import filter_unlabeled_dataframe
from snorkel.utils import preds_to_probs, probs_to_preds

preds_train, probs_train = label_model.predict(L=L_train, return_probs=True)

df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(X=df_train, y=probs_train, L=L_train)
df_train.shape, df_train_filtered.shape


((4458, 2), (1082, 2))